<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/5_Aprendizaje_supervisado/6_Competencia_Seleccion_Mejor_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# 🏆 **Competencia Selección del Mejor Modelo**

**ESTÁ PROHIBIDO EL USO DE GRANDES MODELOS DE LENGUAJE COMO CHATGPT, CLAUDE, GEMINI, ENTRE OTROS, PARA RESOLVER ESTE EJERCICIO**

**Trabajo en parejas o tríos**

**Objetivo:** Predecir las ventas de una compañía (`Sales`) teniendo en cuenta su inversión en publicidad.

**Dataset:** `train_df_ventas_2.csv` disponible en el repositorio del curso.

**IMPORTANTE: Los datos cargados solo corresponden a `train`.**

**Metodología:**
1.  Cargar y explorar los datos.
2.  Preprocesar los datos si es necesario.
3.  De los siguientes modelos, entrenar por lo menos 2:
    - Regresión Lineal
    - Regresión Polinómica
    - KNN Regressor
    - Decision Tree Regressor
    - Random Forest Regressor
    - Gradient Boosting Regressor
    - XGBoost Regressor
4.  Si lo considera necesario, usar `GridSearchCV` con Validación Cruzada (`cv=5`) para optimizar los hiperparámetros. La métrica de optimización debe ser el **RMSE** (Root Mean Squared Error), por lo que debe usar `scoring='neg_root_mean_squared_error'` (el valor será negativo y se multiplicará por -1 al final).
5.  Comparar los modelos y seleccionar el mejor, teniendo en cuenta el menor RMSE.
6. 10 minutos antes de finalizar la clase, enviar el notebook al correo lina.castro6@uexternado.edu.co indicando claramente cuál es el modelo seleccionado. La docente, evaluará este modelo en el `test set`. El equipo que obtenga el menor RMSE ganará la competencia. Si hay empate, ganará quien haya entrenado y evaluado más modelos.

**Premio**: Nota de clase en 5.0 para todos los miembros del equipo ganador.

**Explicación de las variables:**

- Sales: Ventas (millones USD). --> **Esta es la variable objetivo**
- TV: Gasto en promoción televisiva (millones USD).
- Radio: Gasto en promoción radiofónica (millones USD).
- Social Media: Gasto en promoción en redes sociales (millones USD).
- Influencer: Indica si la promoción se realizó en colaboración con Mega, Macro, Nano o Micro influencers.


**Nombres estudiantes del equipo:**

- Juan Diego Moreno Sánchez	
- Juan Esteban Galindo Romero
- Dairon Ferney Reina Sabogal 

# **Desarrollo**

# **Indica claramente cuál modelo seleccionaste como el mejor**

In [73]:
import os
import warnings
warnings.filterwarnings('ignore')

# Manipulación de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesamiento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor  # <--- Versión de Regresión
from sklearn.tree import DecisionTreeRegressor, plot_tree # <--- Versión de Regresión
from sklearn.metrics import mean_squared_error, r2_score # <--- Métricas de Regresión
from sklearn.inspection import permutation_importance # <--- Para Feature Importance
# Métricas y Tuning
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, mean_squared_error

In [24]:
# Que muestre todas las columnas
pd.options.display.max_columns = None
# En los dataframes, mostrar los float con dos decimales
pd.options.display.float_format = '{:,.2f}'.format

# Configuraciones para una mejor visualización
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [25]:
df = pd.read_csv("train_df_ventas_2.csv")
df.head()

,TV,Social Media,Influencer,Radio,Sales
0,17.58,1.22,Macro,22.28,173.61
1,14.59,5.48,Micro,22.10,140.54
2,25.18,2.28,Mega,22.20,197.35
3,12.90,1.83,Nano,21.74,184.05
4,28.38,4.34,Nano,22.61,318.63


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3636 entries, 0 to 3635
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TV            3636 non-null   float64
 1   Social Media  3636 non-null   float64
 2   Influencer    3636 non-null   object 
 3   Radio         3636 non-null   float64
 4   Sales         3636 non-null   float64
dtypes: float64(4), object(1)
memory usage: 142.2+ KB


In [27]:
# 3. Mapear Churn
df['Influencer'] = df['Influencer'].map({'Nano': 0, 'Micro': 1, "Macro": 2, "Mega": 3})

df.head()

,TV,Social Media,Influencer,Radio,Sales
0,17.58,1.22,2,22.28,173.61
1,14.59,5.48,1,22.10,140.54
2,25.18,2.28,3,22.20,197.35
3,12.90,1.83,0,21.74,184.05
4,28.38,4.34,0,22.61,318.63


In [28]:
# Separar X (features) e y (target)
X = df.drop('Sales', axis=1)
y = df['Sales']

In [ ]:
# Identificar columnas numéricas y categóricas
num_cols = [ 'TV', 'Social Media', 'Radio']

# Luego, defines cat_cols como "todas las columnas en X que NO están en num_cols"
cat_cols = [col for col in X.columns if col not in num_cols]

print(f"Columnas Numéricas ({len(num_cols)}): {num_cols}")
print(f"Columnas Categóricas ({len(cat_cols)}): {cat_cols}")

Columnas Numéricas (3): ['TV', 'Social Media', 'Radio']
Columnas Categóricas (1): ['Influencer']


In [33]:
y

0      173.61
1      140.54
2      197.35
3      184.05
4      318.63
        ...  
3631   283.28
3632   266.36
3633    69.73
3634    82.55
3635    36.83
Name: Sales, Length: 3636, dtype: float64

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"\nDistribución de Fuga en y_train: {y_train.mean():.2f}")
print(f"Distribución de Fuga en y_test: {y_test.mean():.2f}")

Forma de X_train: (2908, 4)
Forma de X_test: (728, 4)

Distribución de Fuga en y_train: 193.18
Distribución de Fuga en y_test: 189.31


In [ ]:
# Crear los transformadores
numeric_transformer = StandardScaler()

categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)

# Crear el ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ],
    remainder='passthrough' # Dejar pasar columnas no especificadas (aunque no debería haber)
)


In [36]:
# Ajustar y transformar X_train
X_train_processed = preprocessor.fit_transform(X_train)

# Solo transformar X_test
X_test_processed = preprocessor.transform(X_test)

print("Datos de entrenamiento procesados.")
print(f"Forma de X_train_processed: {X_train_processed.shape}")
print("\nDatos de prueba procesados.")
print(f"Forma de X_test_processed: {X_test_processed.shape}")

# Los modelos ahora usarán X_train_processed y X_test_processed.
# Estos son arrays de NumPy, no DataFrames.

Datos de entrenamiento procesados.
Forma de X_train_processed: (2908, 6)

Datos de prueba procesados.
Forma de X_test_processed: (728, 6)


In [ ]:
print("Iniciando GridSearch para Decision Tree...")
param_grid_dt = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_leaf': [10, 20, 50]
}

grid_dt = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid=param_grid_dt,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1, # Usar todos los procesadores
    verbose=1
)

grid_dt.fit(X_train_processed, y_train)
x = grid_dt.best_score_* -1
print(f"\nMejor Puntuación RMSE en CV: {x:.4f}")
print(f"Mejores Hiperparámetros: {grid_dt.best_params_}")

Iniciando GridSearch para Decision Tree...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Mejor Puntuación RMSE en CV: 1954.4292
Mejores Hiperparámetros: {'max_depth': 5, 'min_samples_leaf': 20}


In [57]:
print("Iniciando GridSearch para Random Forest...")
# NOTA: Reducimos la grilla para que se ejecute más rápido en clase
# En un proyecto real, probaríamos más n_estimators y max_depth
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10]
}

grid_rf = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid=param_grid_rf,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_rf.fit(X_train_processed, y_train)
z = grid_rf.best_score_* -1
print(f"\nMejor Puntuación RMSE en CV: {z:.4f}")
print(f"Mejores Hiperparámetros: {grid_rf.best_params_}")

Iniciando GridSearch para Random Forest...
Fitting 5 folds for each of 4 candidates, totalling 20 fits

Mejor Puntuación RMSE en CV: 1811.2826
Mejores Hiperparámetros: {'max_depth': 5, 'n_estimators': 100}


In [ ]:

print("Iniciando GridSearch para XGBoost...")
param_grid_xgb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1, 0.01],
    'max_depth': [3, 7]
}

grid_xgb = GridSearchCV(
    XGBRegressor(random_state=42,
                  use_label_encoder=False), # ¡El parámetro clave!
    param_grid=param_grid_xgb,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_xgb.fit(X_train_processed, y_train)
w = grid_xgb.best_score_* -1

print(f"\nMejor Puntuación RMSE en CV: {w:.4f}")
print(f"Mejores Hiperparámetros: {grid_xgb.best_params_}")

Iniciando GridSearch para XGBoost...
Fitting 5 folds for each of 12 candidates, totalling 60 fits

Mejor Puntuación RMSE en CV: 1795.1507
Mejores Hiperparámetros: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}


In [72]:
# 1. Re-creamos el pipeline (esto es buena práctica)
knn_pipe_cv = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor())
])

# 2. Definir la grilla de hiperparámetros a probar
# OJO: Usamos 'model__n_neighbors' porque el modelo se llama 'model' dentro del pipeline
param_grid_knn = {
    'model__n_neighbors': [3, 5, 7, 11, 15] # Probaremos estos valores de k
}

# 3. Configurar GridSearchCV
grid_knn = GridSearchCV(
    knn_pipe_cv,
    param_grid_knn,
    cv=5, # 5 folds de validación cruzada
    scoring='neg_mean_squared_error', # Métrica a optimizar: R-cuadrado
    n_jobs=-1, # Usar todos los procesadores
    verbose=1
)

# 4. Ejecutar la búsqueda (sobre los datos de ENTRENAMIENTO)
print("Iniciando GridSearchCV para KNN...")
grid_knn.fit(X_train, y_train)
print("GridSearchCV para KNN completado.")

t = grid_knn.best_score_ * -1

# 5. Mostrar los resultados
print(f"\nMejor hiperparámetro 'k': {grid_knn.best_params_}")
print(f"Mejor score RMSE de validación cruzada: {t:.4f}")

Iniciando GridSearchCV para KNN...
Fitting 5 folds for each of 5 candidates, totalling 25 fits
GridSearchCV para KNN completado.

Mejor hiperparámetro 'k': {'model__n_neighbors': 15}
Mejor score RMSE de validación cruzada: 1979.8935


In [80]:

param_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1, 0.01],
    'max_depth': [3, 7]

}


grid_gb = GridSearchCV(
    GradientBoostingRegressor(random_state=42
                  ), # ¡El parámetro clave!
    param_grid=param_gb,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
# Predecir en el conjunto de prueba

grid_gb.fit(X_train_processed, y_train)
# Evaluar el modelo
v = grid_gb.best_score_* -1
print(f"\nMejor Puntuación RMSE en CV: {v:.4f}")
print(f"Mejores Hiperparámetros: {grid_gb.best_params_}")

Fitting 5 folds for each of 12 candidates, totalling 60 fits

Mejor Puntuación RMSE en CV: 1793.4327
Mejores Hiperparámetros: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100}


El gradiente boosting fue nuestro modelo seleccionado como el mejor, ya que obtuvo el menor RMSE en la evaluación con el conjunto de prueba (test set).